In [2]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#Progress bar
from tqdm import tqdm


#For reading CSV and for math
import numpy as np
import pandas as pd

#Visualize some datasets
import matplotlib.pyplot as plt

#Check our work directory
import os
import shutil


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
photo_folder_path = r'train\train'
csv_file_path = r'train.csv'

In [ ]:
df = pd.read_csv(csv_file_path)

In [ ]:
train_blur = df[df["blur"]==1]
train_sharp = df[df["blur"]==0]

print("blur images:", len(train_blur))
print("sharp images:", len(train_sharp))

In [ ]:
train_val_split = 0.8  # 80% of the photos will be used for training, the rest will be used for validation

In [ ]:
dataset_folder_path = r"Dataset"

In [ ]:
import random

if not os.path.exists(dataset_folder_path):

    if not os.path.exists(os.path.join(dataset_folder_path, 'train', 'blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'train', 'blur'))
    if not os.path.exists(os.path.join(dataset_folder_path, 'train', 'not_blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'train', 'not_blur'))
    if not os.path.exists(os.path.join(dataset_folder_path, 'valid', 'blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'valid', 'blur'))
    if not os.path.exists(os.path.join(dataset_folder_path, 'valid', 'not_blur')):
        os.makedirs(os.path.join(dataset_folder_path, 'valid', 'not_blur'))

    for _, row in df.iterrows():
        filename = row['filename']
        blur = row['blur']
        file_path = os.path.join(photo_folder_path, filename)
        if blur:
            # copy the file to the "train" folder 80% of the time and to the "valid" folder 20% of the time
            if random.random() < train_val_split:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'train', "blur"))
            else:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'valid', "blur"))
        else:
            # copy the file to the "train" folder 80% of the time and to the "valid" folder 20% of the time
            if random.random() < train_val_split:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'train', "not_blur"))
            else:
                shutil.copy(file_path, os.path.join(dataset_folder_path, 'valid', "not_blur"))


In [ ]:
LR = 0.001
BATCH_SIZE = 4
EPOCHS = 15

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [ ]:
train_dir = 'Dataset/train'
val_dir = 'Dataset/valid'

In [ ]:
train_folder = datasets.ImageFolder(root=train_dir, transform=transform)
val_folder = datasets.ImageFolder(root=val_dir,transform=transform)

In [ ]:
train_folder.classes

In [ ]:
train_loader = DataLoader(train_folder,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
val_loader = DataLoader(val_folder,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # input [3, 640, 640]
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),  # [64, 640, 640]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),      # [64, 320, 320]

            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1), # [128, 320, 320]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),      # [128, 160, 160]

            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1), # [256, 160, 160]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),      # [256, 80, 80]

            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1), # [512, 80, 80]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),       # [512, 40, 40]
            
            nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1), # [512, 40, 40]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),       # [512, 20, 20]
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features = 512*20*20, out_features = 1024),
            nn.ReLU(),
            nn.Linear(in_features = 1024, out_features = 512),
            nn.ReLU(),
            nn.Linear(in_features = 512, out_features = 128), 
            nn.ReLU(), 
            nn.Linear(in_features = 128, out_features = 2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)
net = Net()

In [ ]:
for p in net.parameters():
    print(p.shape)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print('Using: ',device)
net.to(device)

Using:  cuda


Net(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14

In [ ]:
print("total allocated memory on gpu (pytorch):", torch.cuda.memory.memory_allocated(device))

In [ ]:
# Cross Entropy Loss  
error = nn.CrossEntropyLoss()

#Adam optimizer
optimizer = optim.Adam(net.parameters(), lr = LR)

In [ ]:
from sklearn.metrics import roc_auc_score

for epoch in range(EPOCHS):
    train_metric = np.array([])
    val_metric = np.array([])
    train_loss = np.array([])
    val_loss = np.array([])

    loop = tqdm(train_loader, leave=True)
    net.train()
    for batch in loop:
        optimizer.zero_grad()
        inputs = batch[0].to(device)
        labels = batch[1].to(device)
        y_pred = net(inputs)

        loss = error(y_pred,labels.long())
        loss.backward()

        loop.set_description(f"Epoch {epoch+1} / {EPOCHS}")
        optimizer.step()
        try:
            auc = roc_auc_score(labels.cpu().numpy(), np.argmax(y_pred.cpu().data.numpy(), axis=1))     # use AUC-ROC metric if different class are represented
            train_metric = np.append(train_metric, auc)
        except:
            train_metric = np.append(train_metric, np.sum(np.argmax(y_pred.cpu().data.numpy(),axis=1) == labels.cpu().numpy()) / BATCH_SIZE)        # else use Accuracy metric
        train_loss = np.append(train_loss, loss.item())
        loop.set_postfix(loss=np.mean(train_loss), AUC_ROC=np.mean(train_metric))
    net.eval()
    with torch.no_grad():
        loop = tqdm(val_loader, leave=True)
        for batch in loop:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            val_pred = net(inputs)

            batch_loss = error(val_pred,labels.long())
            
            loop.set_description(f"Validation {epoch+1}")
            try:
                auc = roc_auc_score(labels.cpu().numpy(), np.argmax(val_pred.cpu().data.numpy(), axis=1))     # use AUC-ROC metric if different class are represented
                val_metric = np.append(val_metric, auc)
            except:
                val_metric = np.append(val_metric, np.sum(np.argmax(val_pred.cpu().data.numpy(),axis=1) == labels.cpu().numpy()) / BATCH_SIZE)        # else use Accuracy metric
            val_loss = np.append(val_loss, batch_loss.item())
            loop.set_postfix(loss=np.mean(val_loss), AUC_ROC=np.mean(val_metric))

In [ ]:
torch.save(net.state_dict(), f"4 model -1 lin layer.pt")

In [4]:
net.load_state_dict(torch.load("2 model lr0.001 e15.pt"))


<All keys matched successfully>

In [5]:
from torch.autograd import Variable
from PIL import Image

def predict_image(image_path, model, device):
    image = Image.open(image_path)

    transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize((640, 640)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze(0)

    if torch.cuda.is_available() and device == "cuda":
        image_tensor.cuda()
        input = Variable(image_tensor.cuda(), requires_grad=True) #torch.Tensor(image_tensor.cuda(), requires_grad=True)
    else:
        input = Variable(image_tensor.cuda(), requires_grad=True) #torch.Tensor(image_tensor.cuda(), requires_grad=True)
    output = model(input)
    #label = output.data.cpu().numpy()
    label = nn.functional.softmax(output.data.cpu())
    label = label.numpy()
    
    return label[0][0].round(1)
    #plt.imshow(image)


In [10]:
predict_image(r"C:\Users\PC\Downloads\pHW-EA3mcr0.jpg", net, device)

C:\Users\PC\AppData\Local\Temp\ipykernel_12168\1854380646.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  label = nn.functional.softmax(output.data.cpu())


0.6

In [3]:
path = r"test\test"
test_imgs = []
for file in os.listdir(path):
    test_imgs.append({'filename': file})
df_test = pd.DataFrame(test_imgs)
print("Test images: ", len(df_test))

Test images:  774


In [ ]:
blurness = []
for file in os.listdir(path):
    blurness.append(predict_image(os.path.join(path, file), net, device))


In [ ]:
blurness

In [ ]:
df_test['blur'] = blurness
df_test['filename'] = df_test['filename'].apply(lambda x : os.path.split(x)[1])
df_test.to_csv('submission 5.csv', index=False)